In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [4]:
df.head()

,target,comment_text
0,0.000000,"This is so cool. It's like, 'would you want yo..."
1,0.000000,Thank you!! This would make my life a lot less...
2,0.000000,This is such an urgent design problem; kudos t...
3,0.000000,Is this something I'll be able to install on m...
4,0.893617,haha you guys are a bunch of losers.


In [5]:
from sklearn.model_selection import train_test_split
X = df['comment_text']
y = df['target']
y = np.where(y>=0.5,1.,0.)
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

In [6]:
unique, counts = np.unique(y_train, return_counts=True)
unique, counts

(array([0., 1.]), array([1162706,  100705]))

In [9]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x_train.values.reshape(-1,1), y_train)
print(sorted(Counter(y_resampled).items()))

[(0.0, 100705), (1.0, 100705)]


In [12]:
X_resampled = X_resampled.reshape(-1)

In [13]:
X_resampled.shape

(201410,)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(X_resampled, y_resampled);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94    497834
         1.0       0.42      0.82      0.56     43629

    accuracy                           0.90    541463
   macro avg       0.70      0.86      0.75    541463
weighted avg       0.94      0.90      0.91    541463

accuracy: 0.895


In [16]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+8.712,idiot
+8.288,idiots
+8.230,stupidity
+8.216,idiotic
+7.306,hypocrite
+6.821,moron
+6.724,morons
+6.379,shit
+6.375,stupid
+5.958,fools


In [17]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

true label: 0.0


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.1


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
pipe_rf = make_pipeline(vec, rf)
pipe_rf.fit(X_resampled, y_resampled)

In [19]:
print_report(pipe_rf, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.98      0.87      0.92    497834
         1.0       0.34      0.78      0.48     43629

    accuracy                           0.86    541463
   macro avg       0.66      0.83      0.70    541463
weighted avg       0.93      0.86      0.89    541463

accuracy: 0.863


In [20]:
vec2 = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.5)
clf2 = LinearSVC()
pipe_tfidf = make_pipeline(vec2, clf2)
pipe_tfidf.fit(X_resampled, y_resampled);

In [21]:
print_report(pipe_tfidf, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93    497834
         1.0       0.38      0.81      0.52     43629

    accuracy                           0.88    541463
   macro avg       0.68      0.85      0.72    541463
weighted avg       0.93      0.88      0.90    541463

accuracy: 0.877


In [22]:
eli5.show_weights(clf2, vec=vec2, top=20)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+10.167,amn
+9.794,hit
+9.257,crap
+8.307,fool
+8.075,suck
+8.057,moro
+7.534,liar
+7.094,fool
+6.842,lown
+6.745,diot


In [23]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf2, row['comment_text'], vec=vec2,))
    print("--"*50)

true label: 0.0


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.5


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


In [24]:
pipe_tfidf_rf = make_pipeline(vec2, rf)
pipe_tfidf_rf.fit(X_resampled, y_resampled);

In [25]:
print_report(pipe_tfidf_rf, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.98      0.87      0.92    497834
         1.0       0.34      0.78      0.47     43629

    accuracy                           0.86    541463
   macro avg       0.66      0.82      0.70    541463
weighted avg       0.93      0.86      0.88    541463

accuracy: 0.860


In [ ]:
y_pred= pipe.predict(df_test)

In [ ]:
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)

In [ ]:
y_pred_2= pipe_tfidf.predict(df_test)

In [ ]:
y_pred_2 = np.where(y_pred_2>=0.5,1.,0.)
sub_df = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
sub_df['prediction'] = y_pred_2
sub_df.to_csv('submission_2.csv',index = False)